<a href="https://colab.research.google.com/github/Rudra-prasad-tarai/CreditRiskOptimisation/blob/main/Credit_Risk_Optimisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Hello")

Hello
